In [32]:
import pandas as pd
import math
import numpy as np
import pyodbc
from sqlalchemy import create_engine
import urllib

In [33]:
level_col = 0
item_col = 1
desc_col = 2
orig_col = 3
check_col = 4
quant_col = 5
unit_col = 6
total_col = 7
rate_col = 7
amount_col = 8

In [34]:
params = urllib.parse.quote_plus('DRIVER={SQL Server};SERVER=WINDOWS-UJU3M60\SQLEXPRESS;DATABASE=SMEC;UID=SMEC;PWD=smecs')
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [35]:
#Read in Excel File to get sheet names
#excel = pd.ExcelFile("Tender Schedule (NEW Branding).xlsm")
excel = pd.ExcelFile("1-1-2.xlsm")
sheets = excel.sheet_names
fname = "1-1-2"
Project_ID = fname.split("-")[0]
Form_ID = fname.split("-")[1]
Version_Number = fname.split("-")[2]

In [36]:
df_sub_form = pd.DataFrame(columns = ["Primary Key","Project ID","Form ID","Version Number","Item ID","Quantity","Price","Unit","Amount"])

In [37]:
df_forms = pd.DataFrame(columns = {"Form ID", "Form Name"})

In [38]:
#Read pages and convert to Data Frames
#Data Frames stored in DF_List
df_list = list()
inc = 0
for x in sheets :
    df_list.append(pd.read_excel("1-1-1.xlsm", sheet_name=x))
    form = [[inc,x]]
    df_forms = df_forms.append(pd.DataFrame(form,columns = ["Form ID", "Form Name"]),ignore_index = True)
    inc += 1

Empty DataFrame
Columns: [Primary Key, Project ID, Form ID, Version Number, Item ID, Quantity, Price, Unit, Amount]
Index: []


In [40]:
# df_col = ["Form ID", "Level 1", "Level 2", "Level 3", "Level 4", "Level 5", "Item ID","Description","Units"]
# df = pd.DataFrame(columns = df_col)
# print(df)

Empty DataFrame
Columns: [Form ID, Level 1, Level 2, Level 3, Level 4, Level 5, Item ID, Description, Units]
Index: []


In [41]:
rows_list = []
for x in range(0,len(sheets)-1):
    inside = False
    for index, rows in df_list[x].iterrows():
        if inside :
            #Check if at end of table
            if rows[total_col] == "Total" :
                inside = False
            else :
                hold = ""
                df_hold = ["","0","0","0","0","0","","",""]
                df_colh = ["","","","","","","","",""]
                full_count = 0
                rows[item_col] = str(rows[item_col])
                if rows[item_col] != "nan":
                    count = 0
                    length = len(rows[item_col])
                    for i in str(rows[item_col]):
                        count += 1
                        if i != ".":
                            hold = hold + i
                            if count == length:
                                df_hold[full_count + 1] = hold
                        else:
                            df_hold[full_count + 1] = hold
                            full_count += 1
                            hold = ""
                    df_hold[7] = rows[desc_col]
                    df_hold[0] = df_forms['Form ID'][x]
                    for y in range(0,6):
                        df_hold[6] = str(df_hold[6]) + str(df_hold[y]) + "."
                    df_hold[6] = df_hold[6][:-1]
                    dict1 = {}
                    dict1.update({"Primary Key": "","Project ID":Project_ID,"Form ID":df_forms['Form ID'][x],"Version Number":Version_Number,"Item ID":df_hold[6],"Quantity":rows[quant_col],"Price":rows[rate_col],"Unit":rows[unit_col],"Amount": rows[amount_col]})
                    rows_list.append(dict1)
                    
                    # df_hold[6] = df_hold[6][:-1]
                    # df_colh[1] = Project_ID
                    # df_colh[2] = df_forms['Form ID'][x]
                    # df_colh[3] = Version_Number
                    # df_colh[4] = df_hold[6]
                    # df_colh[5] = rows[quant_col]
                    # df_colh[6] = rows[rate_col]
                    # df_colh[7] = rows[unit_col]
                    # df_colh[8] = rows[amount_col]
                    # df_hold[8] = rows[unit_col]
                    # s_row = pd.Series(df_hold, index=df.columns)
                    # df_row = pd.Series(df_colh)
                    # df_sub_form = df_sub_form.append(df_colh)
                    # df = df.append(s_row,ignore_index=True)
                    #print(df_colh)       

        if rows[item_col] == "Item" :
            inside = True
df =pd.DataFrame(rows_list)
print(df)

     Primary Key Project ID  Form ID Version Number       Item ID  Quantity  \
0                         1        2              2   2.1.0.0.0.0       NaN   
1                         1        2              2   2.1.1.0.0.0       NaN   
2                         1        2              2   2.1.1.1.0.0       1.0   
3                         1        2              2   2.1.1.2.0.0       1.0   
4                         1        2              2   2.1.2.0.0.0       NaN   
...          ...        ...      ...            ...           ...       ...   
1613                      1       14              2  14.7.3.0.0.0      89.0   
1614                      1       14              2  14.8.0.0.0.0       0.0   
1615                      1       14              2  14.8.1.0.0.0      57.0   
1616                      1       14              2  14.8.2.0.0.0      76.0   
1617                      1       14              2  14.9.0.0.0.0       NaN   

     Price  Unit  Amount  
0      NaN   NaN     NaN

In [43]:
df.to_sql('Submitted_Forms',engine,if_exists='append')

In [44]:
#df.to_sql('Tender Master', engine, if_exists='replace')

In [45]:
#df_forms.to_sql('Form_IDs', engine, if_exists='replace')

In [46]:
#df.to_csv("test.csv")

In [47]:
#df_forms.to_csv("Form_IDs.csv")